In [1]:
#!pip install tweepy

In [2]:
import pandas as pd
import tweepy

auth = tweepy.OAuthHandler('lYSCvNC0HSiLmSTLqTseGGos7', '3XH6USsRkTHsxMbmPqkPmNKpvg8fZqky4SGB7s83HIowt3J9eF')
auth.set_access_token('1310177461007650816-9LFKV2Jvc6x6b82EZyVRJjVVRyiZi3', '1MZRqAwKBOLfDSqwQh79pdGWjVgTiydptVwEbNUnJKL3f')

api = tweepy.API(auth, wait_on_rate_limit=True)

In [39]:
#16 personalities
personalities = ['INFP', 'INFJ', 'INTP', 'INTJ',
                'ENFP', 'ENFJ', 'ENTP', 'ENTJ',
                'ISFP', 'ISFJ', 'ISTP', 'ISTJ',
                'ESFP', 'ESFJ', 'ESTP', 'ESTJ',]

In [40]:
# get users
# search for recently active users

def getUsers(personality):

    users = tweepy.Cursor(api.search_users, q = personality, lang = 'en', since = '2021-01-01', tweet_mode = 'extended').items(300)# .items() all or .items(amount you want)
    
    user_list = []
    userNum = 0
    
    for user in users:
        if user.protected == True:
            continue
        else:
            user_list.append("@" + user.screen_name)
            userNum += 1
        
        if userNum == 100:
            break
            
    if userNum < 100:
        raise ValueError("Not enough users.")
    
    return user_list

def getPersonalityDict(personalities):
    personalityDict = {}
    for personality in personalities:
        personalityDict[personality] = getUsers(personality)
    return personalityDict

personalityDict = getPersonalityDict(personalities)
personalityDict

{'INFP': ['@INFP_Thoughts',
  '@INFPprobs',
  '@BarackOMamba',
  '@infp_things',
  '@LauraAnnTull',
  '@endlessgucci',
  '@YetAnotherINFP',
  '@Infp_Ace',
  '@INF_club',
  '@BaldBeard69',
  '@INFPPrincess',
  '@infpdiary',
  '@JeypsDomingo',
  '@jas_indie',
  '@moodyinfp',
  '@INFP9A',
  '@UmaINFP',
  '@infp_coach',
  '@INFP_Story',
  '@INFP_Thoughts',
  '@INFPprobs',
  '@BarackOMamba',
  '@infp_things',
  '@LauraAnnTull',
  '@endlessgucci',
  '@YetAnotherINFP',
  '@Infp_Ace',
  '@INF_club',
  '@BaldBeard69',
  '@INFPPrincess',
  '@infpdiary',
  '@JeypsDomingo',
  '@jas_indie',
  '@moodyinfp',
  '@INFP9A',
  '@UmaINFP',
  '@infp_coach',
  '@INFP_Story',
  '@jaehyunote',
  '@classicINFP',
  '@infpidk',
  '@Cloud_Lady_INFP',
  '@NindityasNA',
  '@infpbigworld',
  '@Ania_the_INFP',
  '@MoogBassist1971',
  '@INFP_bookworm',
  '@infpindistress',
  '@glynn_w',
  '@infp_soul',
  '@lgbtinfp',
  '@UnhealthyINFp',
  '@naughtydboy',
  '@InfpStan',
  '@infpeniss',
  '@dumpb_tchitis',
  '@RadioFree

In [41]:
# get tweets

def getTweetforUser(user, personality):
    timeline = tweepy.Cursor(api.user_timeline, id = user).items(50)
    
    tweets_df = pd.DataFrame(columns = ['personality', 'date', 'location', 'tweet', 'username', 'bio',
                                    'followers', 'following', 'favourites','retweets','hashtags'])
    
    for tweet in timeline:
    
        tweetdate = tweet.created_at
        location = tweet.user.location
        text = tweet.text
        username = tweet.user.screen_name
        bio = tweet.user.description
        followers = tweet.user.followers_count
        following = tweet.user.friends_count
        favourites = tweet.favorite_count
        retweets = tweet.retweet_count
        
        hashtags_entities = tweet.entities['hashtags']
        #print(hashtags_entities)
        hashtags = []
        
        if len(hashtags_entities) >0:
            for hashtag in hashtags_entities:
                hashtags.append(hashtag['text'])
        
        tweet = [personality, tweetdate, location, text, username, bio, followers, following, favourites, retweets, hashtags]    
        tweets_df.loc[len(tweets_df)] = tweet
        #print(tweets_df)
    return tweets_df

def getTweetforDictionary(personalityDict):

    tweets_df = pd.DataFrame(columns = ['personality', 'date', 'location', 'tweet', 'username', 'bio',
                                    'followers', 'following', 'favourites','retweets','hashtags'])
    
    for personality in personalityDict.keys():
        for user in personalityDict[personality]:
            usertweets = getTweetforUser(user, personality)
            #print(usertweets)
            tweets_df = tweets_df.append(usertweets, ignore_index = True)
   
    return tweets_df


tweets_df = getTweetforDictionary(personalityDict)

In [42]:
tweets_df

,personality,date,location,tweet,username,bio,followers,following,favourites,retweets,hashtags
0,INFP,2021-02-21 10:10:37,Canada,Why is this video SENDING me https://t.co/LPGA...,INFP_Thoughts,Some daily thoughts and opinions of an INFP. 9...,14913,195,0,0,[]
1,INFP,2021-02-20 06:42:52,Canada,"@YanYears Agreeeeee, buttheycanmakezutaraathin...",INFP_Thoughts,Some daily thoughts and opinions of an INFP. 9...,14913,195,0,0,[]
2,INFP,2021-02-20 06:39:21,Canada,@YanYears Some people are speculating that the...,INFP_Thoughts,Some daily thoughts and opinions of an INFP. 9...,14913,195,0,0,[]
3,INFP,2021-02-20 06:35:48,Canada,Why. https://t.co/afvleXpsSi,INFP_Thoughts,Some daily thoughts and opinions of an INFP. 9...,14913,195,21,2,[]
4,INFP,2021-02-14 03:38:39,Canada,@intp_lola God I love Nasir,INFP_Thoughts,Some daily thoughts and opinions of an INFP. 9...,14913,195,1,0,[]
...,...,...,...,...,...,...,...,...,...,...,...
78875,ESTJ,2016-10-09 15:12:23,OXFORD,RT @BishStuart: You get out what you put in @H...,TheGloveManUK,FA Level 2 qualified footy coach. Writer of in...,1276,1937,0,5,[]
78876,ESTJ,2016-10-09 15:10:57,OXFORD,RT @AllistairMcCaw: Great coaching and leaders...,TheGloveManUK,FA Level 2 qualified footy coach. Writer of in...,1276,1937,0,32,[]
78877,ESTJ,2016-10-09 15:10:31,OXFORD,RT @DerbyshireFA: The #Derbyshire Walking Foot...,TheGloveManUK,FA Level 2 qualified footy coach. Writer of in...,1276,1937,0,3,[Derbyshire]
78878,ESTJ,2016-10-09 00:20:39,OXFORD,RT @ShitJokes: Safety meeting at work\n\nThey ...,TheGloveManUK,FA Level 2 qualified footy coach. Writer of in...,1276,1937,0,463,[]


In [43]:
tweets_df.to_csv(r'C:\Users\School\Desktop\4222twitterdata.csv', index = False)
